In [ ]:

# Code for ETL operations on Country-GDP data
# Importing the required libraries
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


#connections

url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
database = 'Banks.db'
table_name = 'Largest_banks'
data_path = 'GDPProj/banks_project'
df = pd.DataFrame(columns=['Name','MC_USD_Billion'])
count = 0
log_file = "log_file.txt" 

#----------------

# --------- with this funtion we locate the table we are looking for

# Print class names of all tables to identify the target table
# this is a test to see home many tasbles athere are and where is the table we are looking for

tables = soup.find_all("table")
for i, table in enumerate(tables):
    print(f"Table {i}: Classes - {table.get('class')}")

    
    
# Attempt to locate the correct table by indexing (assuming it may be the first or second table)

## Based on the output, you may need to adjust the index to target the correct table ##

table = tables[0]  # Change the index based on the output of the above print statements

# After finding out whivj table you need to assign
# you write down the table number ibn the table here!
# very impoetant dont forget


# Verify table selection
print("First few rows of the table:")
for row in table.find("tbody").find_all("tr")[:20]:  # Print first 5 rows to confirm
    print(row)

# Now you may know which which number in your table and what to assign in the functions below

# --------------------------------

# Define the ETL functions
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    # Extraction logic here
    html_page = requests.get(url).text
    soup = BeautifulSoup(html_page, 'html.parser')
    table = soup.find(**table_attribs)
    df = pd.read_html(str(table))[0]
    return df




def transform(df, exchange_rates):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''
    
    # Transformation logic here
    df['GDP_USD_Billion'] = (df['GDP_USD_Million'] / 1000).round(2)
    
    # the exchange rates
    exchange_rates = pd.read_csv('exchange_rate.csv')
    

    # Convert exchange rates to a dictionary for easy lookup
    exchange_dict = exchange_rates.set_index('Currency')['Rate'].to_dict()
    
    # Calculate market cap in different currencies and add to DataFrame
    df_scraped['MC_GBP_Billion'] = (df_scraped['MC_USD_Billion'] * exchange_dict['GBP']).round(2)
    df_scraped['MC_EUR_Billion'] = (df_scraped['MC_USD_Billion'] * exchange_dict['EUR']).round(2)
    df_scraped['MC_INR_Billion'] = (df_scraped['MC_USD_Billion'] * exchange_dict['INR']).round(2)
    
    return df




def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    result = pd.read_sql_query(query_statement, sql_connection)
    print(result)

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''
    with open("log_file.txt", "a") as log_file:
        log_file.write(f"{datetime.now()} - {message}\n")

# Define and call entities/functions to complete the ETL workflow
url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
database = 'Banks.db'
table_name = 'Largest_banks'
csv_path = 'GDPProj/banks_project_output.csv'

# SQL connection
sql_connection = sqlite3.connect(database)

# Execute the ETL process
log_progress("ETL process started.")
df = extract(url, {"name": "table", "class_": "wikitable"})
log_progress("Data extraction completed.")
df = transform(df)
log_progress("Data transformation completed.")
load_to_csv(df, csv_path)
log_progress("Data loaded to CSV.")
load_to_db(df, sql_connection, table_name)
log_progress("Data loaded to database.")

# Running sample queries
run_query("SELECT Name, GDP_USD_Billion FROM Largest_banks WHERE Country = 'USA'", sql_connection)

# Close the SQL connection
sql_connection.close()
log_progress("ETL process completed.")
